In [4]:
!pip install numba

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 39.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/28.1 MB ? eta -:--:--
   ---------------- ----------------------- 11.3/28.1 MB 54.2 MB/s eta 0:00:01
   ---------------------------- ----------- 19.9/28.1 MB 48.4 MB/s eta 0:00:01
   ---------------------------------------- 28.1/28.1 MB 44.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.9 MB ? eta -:--:--
   --------------------------- ------------ 10.7/15.9 MB 51.4 MB/s eta 0:00:01
   ---------------------------------------- 15.9/15.9 MB 45.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.1
    Uninstalling numpy-2.1.1:
      Successfully uninstalled numpy-2.1.1


  You can safely remove it manually.
  You can safely remove it manually.


In [5]:
!pip install dask_ml
!pip install ta
!pip install dill

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 15.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 24.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------- ----------- 8.4/11.6 MB 40.0 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 36.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ------------------------------- -------- 8.7/11.0 MB 44.9 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 38.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.8 MB ? eta -:--:--
   ---- ----------------------------------- 4.7/44.8 MB 25.9 MB/s eta 0:00:02
   ------- -------------------------------- 8.9/44.8 MB 22.2 MB/s eta 0:00:02
   ----------- -------------

In [2]:
!pip install gym


  Using cached gym-0.26.2-py3-none-any.whl
  Using cached gym_notices-0.0.8-py3-none-any.whl.metadata (1.0 kB)
Using cached gym_notices-0.0.8-py3-none-any.whl (3.0 kB)
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ------------------------------ --------- 9.7/12.9 MB 50.2 MB/s eta 0:00:01
   ---------------------------------------- 12.9/12.9 MB 44.9 MB/s eta 0:00:00


In [6]:
!pip install pandas

In [8]:
!pip install tensorflow


  Using cached tensorflow-2.17.0-cp310-cp310-win_amd64.whl.metadata (3.2 kB)
  Using cached tensorflow_intel-2.17.0-cp310-cp310-win_amd64.whl.metadata (5.0 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
Using cached tensorflow-2.17.0-cp310-cp310-win_amd64.whl (2.0 kB)
Using cached tensorflow_intel-2.17.0-cp310-cp310-win_amd64.whl (385.0 MB)
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ---------------------------------------- 4.3/4.3 MB 21.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
   ---------------------------------------- 3.0/3.0 MB 24.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 28.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/26.4 MB ? eta -:--:--
   -------------- ------------------------- 9.

  You can safely remove it manually.


In [9]:
import numpy as np
import pandas as pd
import gym
import dask.dataframe as dd

from gym import spaces
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [11]:
data_full = pd.read_csv('/historique_eth_50lvl.csv')

data_full.fillna(0)

print(data_full.head())
n = len(data_full)




# Liste des colonnes à conserver
colonnes_a_conserver = ['open', 'high', 'low', 'close', 'volume BTC', 'volume USDT']

# Sélectionner les colonnes spécifiques et toutes les colonnes après la 14ème
colonnes_a_conserver.extend(data_full.columns[14:].tolist())

# Sélectionner uniquement les colonnes souhaitées
data_full = data_full[colonnes_a_conserver]



# Définir la taille de l'ensemble d'entraînement

train_size = int(n * 0.8)

# Diviser les données
# Utiliser loc pour sélectionner des sous-ensembles de données
train_data = data_full.loc[:train_size - 1]
test_data = data_full.loc[train_size:]


train_data.to_csv('/content/drive/MyDrive/IA Project/CNN and NEW/DDQN/train_data_multi.csv')
test_data.to_csv('/content/drive/MyDrive/IA Project/CNN and NEW/DDQN/test_data_multi.csv')
#train_data, test_data = train_test_split(data_full, test_size=0.2, shuffle=False)
print(test_data.head())

FileNotFoundError: [Errno 2] No such file or directory: '/historique_eth_50lvl.csv'

###Class

In [6]:
import torch
import collections
import random

class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, next_state, reward, done):
        self.buffer.append((state, action, next_state, reward, done))

    def sample(self, batch_size):
        # Convert the deque to a list before sampling to make it compatible with random.sample()
        batch = random.sample(list(self.buffer), batch_size)
        state, action, next_state, reward, done = zip(*batch)
        return state, action, next_state, reward, done

    def __len__(self):
        return len(self.buffer)

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
import pickle
import dill
from collections import deque
import random
import numba
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer


action_mapping = {"buy": 0, "sell": 1, "hold": 2}

class DoubleDeepQNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, device):
        super(DoubleDeepQNetwork, self).__init__()
        self.device = device
        self.replay_buffer = ReplayBuffer(buffer_size)

        self.encoder_layer = TransformerEncoderLayer(d_model=input_dim, nhead=6, dim_feedforward=hidden_dim, dropout=0.1, batch_first=True).to(device)########################  nhead diviseur de input dim
        self.encoder = TransformerEncoder(self.encoder_layer, num_layers=6).to(device)
        self.fc1 = nn.Linear(input_dim, hidden_dim).to(device)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim).to(device)
        self.fc3 = nn.Linear(hidden_dim, output_dim).to(device)

    def forward(self, x):
        # Assume x is a batch of sequence data with shape (batch_size, input_dim)
        x = x.unsqueeze(0).to(self.device)  # add a dummy dimension to make it compatible with the transformer encoder
        x = x.permute(1, 0, 2).to(self.device)  # (batch_size, input_dim) -> (seq_len, batch_size, input_dim)
        x = self.encoder(x).to(self.device)  # (seq_len, batch_size, input_dim) -> (seq_len, batch_size, input_dim)
        x = x.view(x.size(0), -1).to(self.device)  # flatten the sequence dimension
        x = torch.relu(self.fc1(x)).to(self.device)
        x = torch.relu(self.fc2(x)).to(self.device)
        x = self.fc3(x).to(self.device)
        return x

class DeepDoubleQAgent:
    def __init__(self, input_dim, hidden_dim, output_dim, lr, epsilon, epsilon_decay, buffer_size, device):
        self.online_network = DoubleDeepQNetwork(input_dim, hidden_dim, output_dim, device).to(device)
        self.target_network = DoubleDeepQNetwork(input_dim, hidden_dim, output_dim, device).to(device)
        self.optimizer = optim.Adam(self.online_network.parameters(), lr=lr)
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.gamma = 0.99
        self.device = device
        self.loss_fn = nn.HuberLoss(reduction='mean', delta=1.0)

        self.replay_buffer = ReplayBuffer(buffer_size)


    def select_action(self, state):
      if np.random.rand() < self.epsilon:
          # Random action (exploration)
          actions = ["buy", "sell", "hold"]
          return np.random.choice(actions)
      else:
          with torch.no_grad():
              q_values = self.online_network(state)
              action_idx = torch.argmax(q_values)
              actions = ["buy", "sell", "hold"]
              return actions[action_idx.item()]

    def update_target_network(self):
        self.target_network.load_state_dict(self.online_network.state_dict())



    def update(self, batch_size):
        if len(self.replay_buffer) >= batch_size:
            batch = random.sample(self.replay_buffer, batch_size)
            state, action, next_state, reward, done = zip(*batch)

            # Convert tuples to tensors and move to device
            state = torch.cat(state).to(self.device)
            next_state = torch.cat(next_state).to(self.device)
            action = torch.tensor(action).to(self.device)
            reward = torch.tensor(reward).float().to(self.device)
            done = torch.tensor(done).float().to(self.device)



        ###TEST####

        action_mapping = {'buy': 0, 'sell': 1, 'hold': 2}
        action_numeric = [action_mapping[a] for a in action]

        state = torch.cat(state).to(self.device)
        next_state = torch.cat(next_state).to(self.device)
        # Use the numerical representation of actions
        action = torch.tensor(action_numeric).to(self.device)
        reward = torch.tensor(reward).float().to(self.device)
        done = torch.tensor(done).float().to(self.device)
        ###TEST####

        '''# Ensure all tensors are on the correct device
        state = state.to(self.device)
        next_state = next_state.to(self.device)
        action = action.to(self.device)
        reward = reward.to(self.device)
        done = done.to(self.device)'''

        # Compute Q-values for the current state using the online network
        q_values_online = self.online_network(state)
        q_value_online = q_values_online.gather(1, action.unsqueeze(1)).squeeze(1)

        # Compute Q-values for the next state using the target network (for target values)
        with torch.no_grad():
            next_q_values_online = self.online_network(next_state)
            next_q_values_target = self.target_network(next_state)

            # Double Q-learning: Use online network to select actions, target network to evaluate them
            best_actions = next_q_values_online.argmax(dim=1, keepdim=True)
            next_q_value_target = next_q_values_target.gather(1, best_actions).squeeze(1)

        # Compute target Q-values (using Bellman equation)
        target_q_values = reward + (1 - done) * self.gamma * next_q_value_target

        # Calculate the loss (MSE between online Q-values and target Q-values)
        loss = self.loss_fn(q_value_online, target_q_values)

        # Optimize the online network
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Update the target network (soft update)
        self.update_target_network()

        # Decay epsilon (reduce exploration over time)
        self.epsilon *= self.epsilon_decay # Simplified decay

    def save(self, filename):
        checkpoint = {
            "online_network_state_dict": self.online_network.state_dict(),
            "target_network_state_dict": self.target_network.state_dict(),
            "optimizer_state_dict": self.optimizer.state_dict(),  # Add this line to save optimizer state
            "epsilon": self.epsilon,
            "epsilon_decay": self.epsilon_decay
        }
        torch.save(checkpoint, filename)
        print("Model saved to:", filename)

    @classmethod
    def load(cls, filename):
        checkpoint = torch.load(filename, pickle_module=dill)
        agent = cls(
            input_dim=checkpoint["online_network_state_dict"]["fc1.weight"].shape[1],
            hidden_dim=checkpoint["online_network_state_dict"]["fc1.weight"].shape[0],
            output_dim=checkpoint["online_network_state_dict"]["fc3.weight"].shape[0],
            lr=checkpoint["optimizer_state_dict"]['param_groups'][0]['lr'],  # Load learning rate
            epsilon=checkpoint.get('epsilon', 1.0),  # Load epsilon if saved, otherwise default to 1.0
            epsilon_decay=checkpoint.get('epsilon_decay', 0.99999),  # Load epsilon_decay if saved, otherwise default
            buffer_size=checkpoint.get('buffer_size', 5),  # Load buffer_size if saved, otherwise default
            device=torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use available device
        )
        agent.online_network.load_state_dict(checkpoint["online_network_state_dict"])
        agent.target_network.load_state_dict(checkpoint["target_network_state_dict"])
        agent.optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        return agent
    def sample(self, batch_size):
        indices = np.random.choice(len(self.replay_buffer), batch_size, replace=False)
        batch = [self.replay_buffer[i] for i in indices]
        return batch


    def eval(self):
        self.online_network.eval()
        self.target_network.eval()



###Environnement

In [22]:
import pandas as pd
import numpy as np
import numba


initial_BTC = 1
initial_USDT=0

class TradingEnvironment:
    def __init__(self, csv_file, USDT_balance, BTC_balance):
        self.csv_file = csv_file
        self.USDT_balance = USDT_balance
        self.USDT_balance_prev = USDT_balance
        self.BTC_balance = BTC_balance
        self.BTC_balance_prev = BTC_balance
        self.data = pd.read_csv(csv_file)
        self.current_row = 0
        self.max_value = 0
        self.wallet_BTC=[]
        self.wallet_USDT=[]

    def reset(self):
        self.current_row = 0
        self.USDT_balance = initial_USDT
        self.BTC_balance = initial_BTC
        self.USDT_balance_prev = 0
        self.BTC_balance_prev = 0
        self.old_value = 0

        self.prev_action = 0

        self.wallet_BTC.append(self.BTC_balance)
        self.wallet_USDT.append(self.USDT_balance)

        return self.get_state()


    def step(self, action):

      fees = 0.001  # frais de transactions

      BTC_balance_prev = self.BTC_balance
      USDT_balance_prev = self.USDT_balance

      price = self.data.iloc[self.current_row]["close"]
      price_prev = self.data.iloc[self.current_row -1]["close"]
      current_value = self.BTC_balance * price + self.USDT_balance

      initial_price= self.data.iloc[0]["close"]

      if self.current_row == 0:
        old_price= initial_price
        self.old_value = current_value
      else:
        old_price= self.data.iloc[self.current_row -1]["close"]


      if self.current_row >= len(self.data)-1:


        # We've reached the end of the dataset, stop the episode
        done = True
        print(f"end_value: {current_value},initial_price:{initial_price}, price: {price}")
        reward = 0  # or some other terminal reward
        state = self.get_state()
        return state, reward, done, {}



      if action == "buy" and self.USDT_balance > 0 :
          # Acheter tout BTC avec des USDT
          self.BTC_balance += USDT_balance_prev / price * (1 - fees)
          self.USDT_balance = 0
          self.prev_action = self.current_row ###on actualise la date de la dernière action
      elif action == "sell" and self.BTC_balance > 0:
        # Vendre tout BTC pour des USDT
          self.USDT_balance += BTC_balance_prev * price * (1 - fees)
          self.BTC_balance = 0
          self.prev_action = self.current_row ###on actualise la date de la dernière action
      elif action == "hold":
          # Ne rien faire et conserver les positions actuelles
          pass

      self.current_row += 1

      # Calculate the value of the wallet if no action was taken
      hold_value = (initial_BTC + initial_USDT/initial_price) * price

      if initial_price > price :
        best_value = (initial_BTC + initial_USDT/initial_price * (1-fees)) * price
      else  :
        best_value = initial_BTC * initial_price*(1-fees)+ initial_USDT






      # Calculate the current value of the wallet with the action
      current_value = self.BTC_balance * price + self.USDT_balance

      ################################REWARD############################





      if action== "buy":
        reward = ((1- fees)**2 * (price/price_prev) -1) * 10
      elif action== "sell":
        reward = ((1- fees)**2 * (price_prev/price) -1) * 10
      else:
        reward = 0




      #############################################TEST REWARD####################

      reward += ((1- fees)**2 * (current_value/hold_value)**0.8 -1)

      '''
      if action in ["buy", "sell"]:
        reward -= 0.01 * (1 / (self.current_row-self.prev_action))


      if action == "hold":
        reward = 0.001 * (self.current_row - self.prev_action) * price_prev


      if self.current_row % 3600== 0:

        if self.max_value <= current_value:
          reward += 1000
          self.max_value = current_value
        else:
          reward -= 1000
          self.max_value = current_value
        '''





      ###################################################################
      #print(f"no_action_value: {no_action_value}, current_value: {current_value}")

      #print(f"ETH: {BTC_balance_prev}, USDT: {USDT_balance_prev}, PRICE:{ price}" )

      if self.max_value < current_value:
        self.max_value = current_value
      #elif self.max_value < hold_value:
        #self.max_value = hold_value




      #print(f"reward: {reward}, current_value: {current_value}, no_action_value : {no_action_value}")

      self.wallet_BTC.append(self.BTC_balance)
      self.wallet_USDT.append(self.USDT_balance)

      done = False  # assume the episode is not done unless we've reached the end of the dataset

      return self.get_state(), reward, done, {}

    def get_state(self):
      num_rows = 12
      num_columns = self.data.shape[1]
      start_row = max(0, self.current_row - num_rows + 1)
      end_row = self.current_row + 1
      rows = self.data.iloc[start_row:end_row]

    # Initialize a 2D array with zeros
      state_array = np.zeros((num_rows, num_columns))

    # Fill the array with values from the dataframe, starting from end_row
      state_array[-len(rows):, :] = rows.to_numpy()

      return state_array.reshape((1,-1))



###Train

In [ ]:
# Hyperparameters
input_dim =(87)*12
hidden_dim = 64
output_dim = 3
lr = 8*10**-3
epsilon = 10**(-2)
epsilon_decay = 0.9999

buffer_size = 200
batch_size = 200

n_episode= 30

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Initialize the agent
agent = DeepDoubleQAgent(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, lr=lr, epsilon=epsilon, epsilon_decay=epsilon_decay, buffer_size=buffer_size, device=device)

#Initialize the env

env = TradingEnvironment('/content/drive/MyDrive/IA Project/CNN and NEW/DDQN/train_data_multi.csv', 1000, 0)

# Train the agent
for episode in range(n_episode):
    state = env.reset()
    done = False
    rewards = 0

    while not done:
        # Reshape state to match input_dim
        state_tensor = torch.from_numpy(state).float()  # Add a batch dimension

        # Ensure state has the correct number of features
        if state_tensor.shape[1] != input_dim:
            print(state_tensor.shape)
            print("Warning: State shape does not match input dimension. Check your TradingEnvironment's get_state() method.")
            break  # Exit the current episode

        action = agent.select_action(state_tensor)  # Pass the reshaped tensor
        next_state, reward, done, _ = env.step(action)

        # Reshape next_state as well
        next_state_tensor = torch.from_numpy(next_state).float()

        # Ensure next_state also has the correct number of features
        if next_state_tensor.shape[1] != input_dim:
            print("Warning: Next state shape does not match input dimension. Check your TradingEnvironment's get_state() method. 2")
            break  # Exit the current episode

        # Store the experience in the replay buffer
        agent.replay_buffer.push(state_tensor, action_mapping[action], next_state_tensor, reward, done) # Convert action to numerical using action_mapping

        # Update the agent if there are enough experiences in the buffer
        if len(agent.replay_buffer) > batch_size:
            agent.update(batch_size)  # Pass reshaped tensors

        state = next_state
        rewards += reward
    print(f"Episode {episode+1}, Reward: {rewards}")

end_value: 947.7991669903831,initial_price:3803.0, price: 3455.3
Episode 1, Reward: -28802.91102616713



###SAVE

In [ ]:
agent.save("trained_model_best.zip")

In [ ]:
agent = agent.load("trained_model.zip")

###TEST loaded agent

In [ ]:
test_env = TradingEnvironment('/content/drive/MyDrive/IA Project/CNN and NEW/DDQN/test_data_multi.csv', 1000, 0)

In [ ]:


# Set the agent to evaluation mode
agent.eval()

# Test the agent on a single episode
state = test_env.reset()
done = False
rewards = 0

while not done:
    # Reshape state to match input_dim
    state_tensor = torch.from_numpy(state).float()  # Add a batch dimension

    # Ensure state has the correct number of features
    if state_tensor.shape[1] != input_dim:
      print(state_tensor.shape)
      print(input_dim)
      print("Warning: State shape does not match input dimension. Check your TradingEnvironment's get_state() method.")
      break  # Exit the episode

    action = agent.select_action(state_tensor)  # Pass the reshaped tensor
    next_state, reward, done, _ = env.step(action)

    # Reshape next_state as well
    next_state_tensor = torch.from_numpy(next_state).float()

    # Ensure next_state also has the correct number of features
    if next_state_tensor.shape[1] != input_dim:
        print("Warning: Next state shape does not match input dimension. Check your TradingEnvironment's get_state() method. 2")
        break  # Exit the episode

    state = next_state
    rewards += reward

print(f"Final Reward: {rewards}")

In [ ]:
import numba.cuda as cuda

if cuda.is_available():
    print("CUDA is available!")
    print("Number of CUDA devices:", cuda.gpus)
else:
    print("CUDA is not available. Please check your installation.")

CUDA is available!
Number of CUDA devices: <Managed Device 0>


In [ ]:
!pip install --upgrade numba

import numpy as np
from numba import cuda


@cuda.jit
def gpu_accelerated_function(data):
    idx = cuda.grid(1)  # Get the global thread index in 1D
    if idx < data.size:
        data[idx] *= cuda.blockIdx.x
    return

# Create a sample dataset
data = np.random.rand(1000)

# Move data to the GPU
data_gpu = cuda.to_device(data)

# Define the grid and block dimensions
threads_per_block = 128
blocks_per_grid = (data.size + (threads_per_block - 1)) // threads_per_block

# Perform computations on the GPU
gpu_accelerated_function[blocks_per_grid, threads_per_block](data_gpu)

# Move results back to the CPU
result_cpu = data_gpu.copy_to_host()

print(result_cpu)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 8 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.000000